# Notebook to generate psychometrics plot given sessions & conditions

In [1]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()


Local configuration file found !!, no need to run the configuration (unless configuration has changed)


In [8]:
import datajoint as dj
import pandas as pd
import numpy as np

import u19_pipeline.utils.path_utils as pu

from matplotlib import cm
from u19_pipeline import utility
import pathlib
import pickle

### Datajoint configuration and Connection to DB

In [9]:

dj.conn()
acquisition_db = dj.create_virtual_module('acquisition', 'u19_acquisition')
pupillometry_db = dj.create_virtual_module('pupillometry', 'u19_pupillometry')

In [10]:
root_videopath = dj.config['custom']['dlc_root_data_dir']

modelPath = pathlib.Path(root_videopath, 'MODELS', 'Pupillometry2-Ryan-2022-04-07').as_posix()
configPath = pathlib.Path(modelPath,'config.yaml').as_posix()

proccesed_rootpath = dj.config['custom']['dlc_processed_data_dir']
proccesed_rootpath = pathlib.Path(proccesed_rootpath, 'pupillometry').as_posix()
configPath

'/mnt/cup/braininit/Data/Raw/video_acquisition/MODELS/Pupillometry2-Ryan-2022-04-07/config.yaml'

In [11]:
key = 'subject_fullname like "jounhong%"'
video_data_df = pd.DataFrame((pupillometry_db.PupillometrySession * acquisition_db.SessionVideo & key).fetch("KEY", 'remote_path_video_file', as_dict=True))

video_data_df['remote_path_video_file'] = 'pupillometry/' + video_data_df['remote_path_video_file']
video_data_df.loc[0,'remote_path_video_file']

video_data_df['processed_path'] = video_data_df['remote_path_video_file'].apply(lambda x : pathlib.Path(x).parent.as_posix())

video_data_df

,subject_fullname,session_date,session_number,video_type,remote_path_video_file,processed_path
0,jounhong_TH_276,2022-04-01,0,pupillometry,pupillometry/jounhong/jounhong_TH_276/20220401...,pupillometry/jounhong/jounhong_TH_276/20220401_g0
1,jounhong_TH_276,2022-04-02,0,pupillometry,pupillometry/jounhong/jounhong_TH_276/20220402...,pupillometry/jounhong/jounhong_TH_276/20220402_g0
2,jounhong_TH_276,2022-04-03,0,pupillometry,pupillometry/jounhong/jounhong_TH_276/20220403...,pupillometry/jounhong/jounhong_TH_276/20220403_g0
3,jounhong_TH_276,2022-04-04,0,pupillometry,pupillometry/jounhong/jounhong_TH_276/20220404...,pupillometry/jounhong/jounhong_TH_276/20220404_g0
4,jounhong_TH_276,2022-04-05,0,pupillometry,pupillometry/jounhong/jounhong_TH_276/20220405...,pupillometry/jounhong/jounhong_TH_276/20220405_g0
5,jounhong_TH_276,2022-04-06,0,pupillometry,pupillometry/jounhong/jounhong_TH_276/20220406...,pupillometry/jounhong/jounhong_TH_276/20220406_g0
6,jounhong_TH_276,2022-04-07,0,pupillometry,pupillometry/jounhong/jounhong_TH_276/20220407...,pupillometry/jounhong/jounhong_TH_276/20220407_g0
7,jounhong_TH_276,2022-04-08,0,pupillometry,pupillometry/jounhong/jounhong_TH_276/20220408...,pupillometry/jounhong/jounhong_TH_276/20220408_g0
8,jounhong_TH_276,2022-04-11,0,pupillometry,pupillometry/jounhong/jounhong_TH_276/20220411...,pupillometry/jounhong/jounhong_TH_276/20220411_g0
9,jounhong_TH_276,2022-04-12,0,pupillometry,pupillometry/jounhong/jounhong_TH_276/20220412...,pupillometry/jounhong/jounhong_TH_276/20220412_g0


## Filter params

In [18]:

for i in range(video_data_df.shape[0]):
    this_video_path = pathlib.Path(root_videopath, video_data_df.loc[i, 'remote_path_video_file']).as_posix()
    this_results_path = pathlib.Path(proccesed_rootpath, video_data_df.loc[i, 'processed_path']).as_posix()

    pickle_file = pu.get_filepattern_paths(this_results_path, "/*pupil_diameter.pickle")

    print(pickle_file)

    if len(pickle_file) > 0:
        pickle_file = pickle_file[0]
        
        objects = []
        with (open(pickle_file, "rb")) as openfile:
            while True:
                try:
                    object = pickle.load(openfile)
                except EOFError:
                    break


        pupillometry_data_array = object

        print(type(pupillometry_data_array))
        print(pupillometry_data_array)

        key = dict()
        key['subject_fullname'] = video_data_df.loc[i, 'subject_fullname']
        key['session_date'] = video_data_df.loc[i, 'session_date']
        new_key = (acquisition_db.Session & key).fetch1("KEY")
        new_key['pupil_diameter'] = pupillometry_data_array
        pupillometry_db.PupillometryData.insert1(new_key, allow_direct_insert=True)


['/mnt/cup/braininit/Data/Processed/video_acquisition/pupillometry/pupillometry/jounhong/jounhong_TH_276/20220401_g0/pupil_diameter.pickle']
<class 'numpy.ndarray'>
[24.63723836 24.29144872 24.24534885 ... 37.03335293 37.11923868
 37.21675184]
['/mnt/cup/braininit/Data/Processed/video_acquisition/pupillometry/pupillometry/jounhong/jounhong_TH_276/20220402_g0/pupil_diameter.pickle']
<class 'numpy.ndarray'>
[10.94436963 10.94436401 10.94436869 ... 10.94436995 10.94451484
 10.94450834]
['/mnt/cup/braininit/Data/Processed/video_acquisition/pupillometry/pupillometry/jounhong/jounhong_TH_276/20220403_g0/pupil_diameter.pickle']
<class 'numpy.ndarray'>
[10.94436515 10.94436729 10.9443641  ... 10.94436633 10.94451562
 10.94436764]
[]
[]
[]
['/mnt/cup/braininit/Data/Processed/video_acquisition/pupillometry/pupillometry/jounhong/jounhong_TH_276/20220407_g0/pupil_diameter.pickle']
<class 'numpy.ndarray'>
[        nan         nan         nan ... 38.84354716 38.76662294
 39.17393913]
['/mnt/cup/brai